In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import random
import nltk
import re


import matplotlib.pyplot as plt
from ipywidgets import interactive

from seaborn import lmplot, stripplot

#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

In [3]:
np.random.seed(500)

In [4]:
Corpus = pd.read_csv("Data_hudoc_cases.csv",encoding='latin-1')

In [5]:
Corpus.head()

,url,language,title,JUDGMENT,introduction,Facts,The Law,releavent legal framework,Unanimously,courts_assesment,Admissibility
0,"https://hudoc.echr.coe.int/eng/#{""tabview"":[""d...",English,CASE OF NEGOVANOVIÄ AND OTHERS v. SERBIA (App...,Art 1 P12 â¢ Prohibition of discrimination â...,"1., , ,The applications concern alleged discri...","2,., , ,A list of the applicants is set out in...",",joinder of the applications,43., , ,H...","The ,Sporting Achievements Recognition and Rew...",",Decides,, unanimously, to join the applicati...",NaN,"(a), ,The Courtâs jurisdiction ,ratione ma..."
1,"https://hudoc.echr.coe.int/eng/#{""tabview"":[""d...",English,CASE OF BEG S.P.A. v. ITALY (Application no. ...,",Art 6 Â§ 1 (civil) â¢ Impartial tribunal â...","1., , ,The case concerns, under Article 6 Â§ 1...","2., , ,The applicant is an Italian company, wh...",NaN,"Relevant domestic law,38., , ,The Court of Cas...","Declares, the application admissible;,Holds, t...","125., , ,The Court reiterates that Article 6 Â...","Whether the Court has jurisdiction ,ratione pe..."
2,"https://hudoc.echr.coe.int/eng#{""itemid"":[""001...",English,CASE OF ADVANCE PHARMA SP. Z O.O v. POLAND (Ap...,Art 6 (civil) â Tribunal established by law ...,"1., , ,The applicant company had its cassation...",NaN,NaN,NaN,"Joins ,to the merits the Governmentâs prelim...",NaN,"Exhaustion of domestic remedies,(a), ,The pa..."
3,"https://hudoc.echr.coe.int/eng#{""itemid"":[""001...",English,CASE OF KOMISSAROV v. THE CZECH REPUBLIC (Appl...,",Art 5 Â§ 1 (f) â¢ Extradition â¢ Excessive...","1., , ,The application concerns in particular ...","2., , ,The applicant was born in 1968. He live...","PRELIMINARY REMARKS,34., , ,In his claims for ...",LAW NO. 104/2013 ON INTERNATIONAL JUDICIAL COO...,"Declares, the complaint under Article 5 Â§ 1 (...",NaN,"40., , ,The Government accepted that the appli..."
4,"https://hudoc.echr.coe.int/eng#{""itemid"":[""001...",English,CASE OF Å EKS v. CROATIA (Application no. 393...,",Art 10 â¢ Freedom to impart information â¢...","1., , ,The applicant requested access to the c...","2., , ,The applicant was born in 1943 and live...",ALLEGED VIOLATION OF ARTICLE 10 OF THE CONVENT...,"Relevant domestic law,20., , ,The relevant pro...","Declares, the application admissible;,Holds, t...",NaN,"Compatibility ,ratione materiae,(a), ,The pa..."


In [6]:
Corpus.shape

(12271, 11)

In [7]:
Corpus.columns

Index(['url', 'language', 'title', 'JUDGMENT', 'introduction', 'Facts',
       'The Law', 'releavent legal framework', 'Unanimously',
       'courts_assesment', 'Admissibility'],
      dtype='object')

In [8]:
Corpus.dtypes

url                          object
language                     object
title                        object
JUDGMENT                     object
introduction                 object
Facts                        object
The Law                      object
releavent legal framework    object
Unanimously                  object
courts_assesment             object
Admissibility                object
dtype: object

In [9]:
Corpus.describe(include="all")


,url,language,title,JUDGMENT,introduction,Facts,The Law,releavent legal framework,Unanimously,courts_assesment,Admissibility
count,11214,12196,11078,9796,4910,7094,8806,4515,9910,3215,6985
unique,4601,17,2201,1063,480,1298,2245,450,2167,724,1542
top,"https://hudoc.echr.coe.int/eng/#{""tabview"":[""d...",English,CASE OF,",","1., , ,The present case concerns the death of ...","2., , ,The first applicant is an association r...",ALLEGED VIOLATION OF ARTICLE 6 OF THE CONVENTI...,"22,., , ,The relevant provisions of the Crimin...","Declares, the application admissible;,Holds, t...","90., , ,It is now well established that althou...","31., , ,The Court notes that the application i..."
freq,15,11582,885,1812,45,28,24,28,40,24,36


In [10]:
Corpus['url'].dropna(how='all', inplace=True)

Corpus = Corpus.applymap(str)

In [11]:
Corpus.head()

,url,language,title,JUDGMENT,introduction,Facts,The Law,releavent legal framework,Unanimously,courts_assesment,Admissibility
0,"https://hudoc.echr.coe.int/eng/#{""tabview"":[""d...",English,CASE OF NEGOVANOVIÄ AND OTHERS v. SERBIA (App...,Art 1 P12 â¢ Prohibition of discrimination â...,"1., , ,The applications concern alleged discri...","2,., , ,A list of the applicants is set out in...",",joinder of the applications,43., , ,H...","The ,Sporting Achievements Recognition and Rew...",",Decides,, unanimously, to join the applicati...",nan,"(a), ,The Courtâs jurisdiction ,ratione ma..."
1,"https://hudoc.echr.coe.int/eng/#{""tabview"":[""d...",English,CASE OF BEG S.P.A. v. ITALY (Application no. ...,",Art 6 Â§ 1 (civil) â¢ Impartial tribunal â...","1., , ,The case concerns, under Article 6 Â§ 1...","2., , ,The applicant is an Italian company, wh...",nan,"Relevant domestic law,38., , ,The Court of Cas...","Declares, the application admissible;,Holds, t...","125., , ,The Court reiterates that Article 6 Â...","Whether the Court has jurisdiction ,ratione pe..."
2,"https://hudoc.echr.coe.int/eng#{""itemid"":[""001...",English,CASE OF ADVANCE PHARMA SP. Z O.O v. POLAND (Ap...,Art 6 (civil) â Tribunal established by law ...,"1., , ,The applicant company had its cassation...",nan,nan,nan,"Joins ,to the merits the Governmentâs prelim...",nan,"Exhaustion of domestic remedies,(a), ,The pa..."
3,"https://hudoc.echr.coe.int/eng#{""itemid"":[""001...",English,CASE OF KOMISSAROV v. THE CZECH REPUBLIC (Appl...,",Art 5 Â§ 1 (f) â¢ Extradition â¢ Excessive...","1., , ,The application concerns in particular ...","2., , ,The applicant was born in 1968. He live...","PRELIMINARY REMARKS,34., , ,In his claims for ...",LAW NO. 104/2013 ON INTERNATIONAL JUDICIAL COO...,"Declares, the complaint under Article 5 Â§ 1 (...",nan,"40., , ,The Government accepted that the appli..."
4,"https://hudoc.echr.coe.int/eng#{""itemid"":[""001...",English,CASE OF Å EKS v. CROATIA (Application no. 393...,",Art 10 â¢ Freedom to impart information â¢...","1., , ,The applicant requested access to the c...","2., , ,The applicant was born in 1943 and live...",ALLEGED VIOLATION OF ARTICLE 10 OF THE CONVENT...,"Relevant domestic law,20., , ,The relevant pro...","Declares, the application admissible;,Holds, t...",nan,"Compatibility ,ratione materiae,(a), ,The pa..."


In [12]:
print(Corpus.dropna().shape)

(12271, 11)


In [13]:
Corpus['label'] = ''
for i,x in Corpus.Unanimously.iteritems():
    is_violation = re.search('has been a violation of', x)
    if is_violation:
        Corpus['label'][i] = 1
    else:
        Corpus['label'][i] = 0

In [14]:
Corpus.label.mean()

0.6129899763670442

In [15]:
Corpus.head(50)


,url,language,title,JUDGMENT,introduction,Facts,The Law,releavent legal framework,Unanimously,courts_assesment,Admissibility,label
0,"https://hudoc.echr.coe.int/eng/#{""tabview"":[""d...",English,CASE OF NEGOVANOVIÄ AND OTHERS v. SERBIA (App...,Art 1 P12 â¢ Prohibition of discrimination â...,"1., , ,The applications concern alleged discri...","2,., , ,A list of the applicants is set out in...",",joinder of the applications,43., , ,H...","The ,Sporting Achievements Recognition and Rew...",",Decides,, unanimously, to join the applicati...",nan,"(a), ,The Courtâs jurisdiction ,ratione ma...",1
1,"https://hudoc.echr.coe.int/eng/#{""tabview"":[""d...",English,CASE OF BEG S.P.A. v. ITALY (Application no. ...,",Art 6 Â§ 1 (civil) â¢ Impartial tribunal â...","1., , ,The case concerns, under Article 6 Â§ 1...","2., , ,The applicant is an Italian company, wh...",nan,"Relevant domestic law,38., , ,The Court of Cas...","Declares, the application admissible;,Holds, t...","125., , ,The Court reiterates that Article 6 Â...","Whether the Court has jurisdiction ,ratione pe...",1
2,"https://hudoc.echr.coe.int/eng#{""itemid"":[""001...",English,CASE OF ADVANCE PHARMA SP. Z O.O v. POLAND (Ap...,Art 6 (civil) â Tribunal established by law ...,"1., , ,The applicant company had its cassation...",nan,nan,nan,"Joins ,to the merits the Governmentâs prelim...",nan,"Exhaustion of domestic remedies,(a), ,The pa...",1
3,"https://hudoc.echr.coe.int/eng#{""itemid"":[""001...",English,CASE OF KOMISSAROV v. THE CZECH REPUBLIC (Appl...,",Art 5 Â§ 1 (f) â¢ Extradition â¢ Excessive...","1., , ,The application concerns in particular ...","2., , ,The applicant was born in 1968. He live...","PRELIMINARY REMARKS,34., , ,In his claims for ...",LAW NO. 104/2013 ON INTERNATIONAL JUDICIAL COO...,"Declares, the complaint under Article 5 Â§ 1 (...",nan,"40., , ,The Government accepted that the appli...",1
4,"https://hudoc.echr.coe.int/eng#{""itemid"":[""001...",English,CASE OF Å EKS v. CROATIA (Application no. 393...,",Art 10 â¢ Freedom to impart information â¢...","1., , ,The applicant requested access to the c...","2., , ,The applicant was born in 1943 and live...",ALLEGED VIOLATION OF ARTICLE 10 OF THE CONVENT...,"Relevant domestic law,20., , ,The relevant pro...","Declares, the application admissible;,Holds, t...",nan,"Compatibility ,ratione materiae,(a), ,The pa...",0
5,"https://hudoc.echr.coe.int/eng/#{""tabview"":[""d...",English,CASE OF DENIS AND IRVINE v. BELGIUM (Applicati...,Art 5 Â§ 1 â¢ Lawful arrest or detention â¢ ...,"1., , ,The case concerns the alleged unlawfuln...",nan,nan,nan,"Upholds,, unanimously, the Governmentâs prel...","98., , ,The Court reiterates that the âcaseâ...",nan,0
6,"https://hudoc.echr.coe.int/eng#{""itemid"":[""001...",English,CASE OF BUSUTTIL v. MALTA (Application no. 48...,"Art 6 Â§ 2 â¢ Presumption of innocence â¢, ,...","1., , ,The case concerns criminal proceedings ...","2., , ,The applicant was born in 1947 and live...",ALLEGED VIOLATION OF ARTICLE 6 Â§ 2 OF THE CON...,"DOMESTIC LAW,28,., , ,Section 23 sub,â,parag...","Declares, the application admissible;,Holds, t...","46,., , ,The Court reiterates that viewed as a...","34., , ,The Court notes that the complaint is ...",0
7,"https://hudoc.echr.coe.int/eng/#{""tabview"":[""d...",English,CASE OF KURT v. AUSTRIA (Application no. 6290...,",Art 2 (substantive) â¢ Positive obligations...",nan,"10., , ,The applicant was born in 1978 and liv...",nan,nan,"Declares,, unanimously, the complaint concerni...",nan,nan,0
8,"https://hudoc.echr.coe.int/eng/#{""tabview"":[""d...",English,CASE OF SÃNDOR VARGA AND OTHERS v. HUNGARY (A...,",Art 3 (substantive) â¢ Inhuman or degrading...","1., , ,The cases concern the applicantsâ lif...","2., , ,The applicantsâ details are set out i...","JOINDER OF THE APPLICATIONS,21., , ,Having reg...","13,., , ,The Fundamental Law of Hungary provid...",",Decides ,to join the applications;,Declares ...",nan,"Exhaustion of domestic rem

In [16]:
Corpus.dtypes


url                          object
language                     object
title                        object
JUDGMENT                     object
introduction                 object
Facts                        object
The Law                      object
releavent legal framework    object
Unanimously                  object
courts_assesment             object
Admissibility                object
label                        object
dtype: object

In [17]:
Corpus.label.mean()

0.6129899763670442

In [21]:
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['Facts'] = [entry.lower() for entry in Corpus['Facts']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['Facts']= [word_tokenize(entry) for entry in Corpus['Facts']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['Facts']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

AttributeError: 'list' object has no attribute 'lower'

In [19]:
Corpus.head()

,url,language,title,JUDGMENT,introduction,Facts,The Law,releavent legal framework,Unanimously,courts_assesment,Admissibility,label,text_final
0,"https://hudoc.echr.coe.int/eng/#{""tabview"":[""d...",English,CASE OF NEGOVANOVIÄ AND OTHERS v. SERBIA (App...,Art 1 P12 â¢ Prohibition of discrimination â...,"1., , ,The applications concern alleged discri...","[2, ,, ., ,, ,, ,, a, list, of, the, applicant...",",joinder of the applications,43., , ,H...","The ,Sporting Achievements Recognition and Rew...",",Decides,, unanimously, to join the applicati...",nan,"(a), ,The Courtâs jurisdiction ,ratione ma...",1,"['list', 'applicant', 'set', 'appendix', 'pers..."
1,"https://hudoc.echr.coe.int/eng/#{""tabview"":[""d...",English,CASE OF BEG S.P.A. v. ITALY (Application no. ...,",Art 6 Â§ 1 (civil) â¢ Impartial tribunal â...","1., , ,The case concerns, under Article 6 Â§ 1...","[2., ,, ,, ,, the, applicant, is, an, italian,...",nan,"Relevant domestic law,38., , ,The Court of Cas...","Declares, the application admissible;,Holds, t...","125., , ,The Court reiterates that Article 6 Â...","Whether the Court has jurisdiction ,ratione pe...",1,"['applicant', 'italian', 'company', 'represent..."
2,"https://hudoc.echr.coe.int/eng#{""itemid"":[""001...",English,CASE OF ADVANCE PHARMA SP. Z O.O v. POLAND (Ap...,Art 6 (civil) â Tribunal established by law ...,"1., , ,The applicant company had its cassation...",[nan],nan,nan,"Joins ,to the merits the Governmentâs prelim...",nan,"Exhaustion of domestic remedies,(a), ,The pa...",1,['nan']
3,"https://hudoc.echr.coe.int/eng#{""itemid"":[""001...",English,CASE OF KOMISSAROV v. THE CZECH REPUBLIC (Appl...,",Art 5 Â§ 1 (f) â¢ Extradition â¢ Excessive...","1., , ,The application concerns in particular ...","[2., ,, ,, ,, the, applicant, was, born, in, 1...","PRELIMINARY REMARKS,34., , ,In his claims for ...",LAW NO. 104/2013 ON INTERNATIONAL JUDICIAL COO...,"Declares, the complaint under Article 5 Â§ 1 (...",nan,"40., , ,The Government accepted that the appli...",1,"['applicant', 'bear', 'live', 'nizhny', 'novgo..."
4,"https://hudoc.echr.coe.int/eng#{""itemid"":[""001...",English,CASE OF Å EKS v. CROATIA (Application no. 393...,",Art 10 â¢ Freedom to impart information â¢...","1., , ,The applicant requested access to the c...","[2., ,, ,, ,, the, applicant, was, born, in, 1...",ALLEGED VIOLATION OF ARTICLE 10 OF THE CONVENT...,"Relevant domestic law,20., , ,The relevant pro...","Declares, the application admissible;,Holds, t...",nan,"Compatibility ,ratione materiae,(a), ,The pa...",0,"['applicant', 'bear', 'live', 'zagreb', 'repre..."


In [20]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)

print(Train_Y)


Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

print(Train_Y)

Tfidf_vect = TfidfVectorizer(max_features=1000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

print(Tfidf_vect.vocabulary)

# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print(confusion_matrix(Test_Y, predictions_SVM))

4713     1
8113     1
4121     1
3464     1
6138     0
        ..
3005     0
3790     1
8465     1
12097    1
951      0
Name: label, Length: 8589, dtype: object
[1 1 1 ... 1 1 0]
None
SVM Accuracy Score ->  79.60347637153721
[[1321  113]
 [ 638 1610]]
